In [1]:
import nglview as nv
import openmm
import openmm.app as app
import openmm.unit as unit
import pytraj as pt
from tqdm.notebook import tqdm

In [2]:
work_folder = "simulation"

### Load Files

In [3]:
pdbfile = app.PDBFile(f"{work_folder}/system.pdb")
with open(f"{work_folder}/system.xml", "r") as f:
    system = openmm.XmlSerializer.deserialize(f.read())

### Configure simulation

In [4]:
integrator = openmm.LangevinIntegrator(
    298.15 * unit.kelvin, 1.0 / unit.picosecond, 2.0 * unit.femtosecond
)

In [5]:
simulation = app.Simulation(
    pdbfile.topology,
    system,
    integrator,
    openmm.Platform.getPlatformByName("CUDA"),
    {"CudaPrecision": "mixed", "CudaDeviceIndex": "0"},
)
simulation.context.setPositions(pdbfile.positions)

dcd_reporter = app.DCDReporter(f"{work_folder}/equilibration.dcd", 500)
state_reporter = app.StateDataReporter(
    f"{work_folder}/equilibration.log",
    500,
    step=True,
    kineticEnergy=True,
    potentialEnergy=True,
    totalEnergy=True,
    temperature=True,
    speed=True,
    separator=",",
)
simulation.reporters.append(dcd_reporter)
simulation.reporters.append(state_reporter)

### Initial Equilibration

In [6]:
simulation.minimizeEnergy()

simulation.step(50000)

### Let it Grow! Either by increasing Radius or Lambda

In [7]:
# Growing radius
r_particle = [0.5, 1.0, 1.5, 2.0, 2.5, 3.0, 3.5, 4.0, 4.5, 5.0] * unit.angstrom

for radius in tqdm(r_particle):
    simulation.context.setParameter("R_particle", radius)
    simulation.step(50000)

  0%|          | 0/10 [00:00<?, ?it/s]

In [8]:
# # Growing lambda
# r_particle = 5.0 * unit.angstrom
# simulation.context.setParameter("R_particle", r_particle)

# lambda_values = [0.0, 0.2, 0.4, 0.6, 0.8, 1.0]

# for lb in tqdm(lambda_values):
#     simulation.context.setParameter("lambda_sterics", lb)
#     simulation.step(50000)

In [9]:
traj = pt.iterload(f"{work_folder}/equilibration.dcd", top=f"{work_folder}/system.pdb")
pt.align(traj, mask="@/C")

pytraj.TrajectoryIterator, 1100 frames: 
Size: 0.003639 (GB)
<Topology: 148 atoms, 2 residues, 2 mols, non-PBC>
           

In [11]:
view = nv.show_pytraj(traj)
view

NGLWidget(max_frame=1099)